In [ ]:
%matplotlib widget

# загрузим данные
import numpy as np
import pandas as pd

file_path = "../npz/BTCUSD_1T_close_only.npz"
data = np.load(file_path, allow_pickle=True)
df = pd.DataFrame(data['data'], columns=['timestamp', 'close'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

In [ ]:
dfc = df[:10000].copy()

In [ ]:
lim = 1000
def get_transitions_map(df, windows, tau, grid_sizes, transitions_grid_size):
    """
    creates a map of price shifts
    """
    max_window = max(windows)
    mas = {}
    diffs = {}
    closes = df['close']
    transition_values = closes.shift(-tau) - closes
    transitions_min = transition_values.min()
    transitions_max = transition_values.max()
    for wi, w in enumerate(windows):
        mas[wi] = closes.rolling(w).mean()
        diffs[wi] = mas[wi] - mas[wi - 1] if wi > 0 else mas[0] - df['close']
    diff_mins = []
    diff_maxs = []
    for wi, w in enumerate(windows):
        diff_mins.append(diffs[wi].min())
        diff_maxs.append(diffs[wi].max())
    transitions = {}
    transitions_map = {}
    for i in range(max_window, max_window + lim):
        positions = []
        for wi, w in enumerate(windows):
            diff = diffs[wi].iloc[i]
            diff_min = diff_mins[wi]
            diff_max = diff_maxs[wi]
            grid_size = grid_sizes[wi]
            pos = round((grid_size - 1) * (diff - diff_min) / (diff_max - diff_min))
            pos_fixed = max(0, min(pos, grid_size - 1))
            positions.append(pos_fixed)
        transitions[tuple(positions)] = transition_values.iloc[i]
        transitions_pos = round((transitions_grid_size - 1) * (transition_values.iloc[i] - transitions_min) / (transitions_max - transitions_min))
        transitions_pos_fixed = max(0, min(transitions_pos, transitions_grid_size - 1))
        if (transitions_map.get(tuple(positions), None) is None):
            transitions_map[tuple(positions)] = {}
        if (transitions_map[tuple(positions)].get(transitions_pos_fixed, None) is None):
            transitions_map[tuple(positions)][transitions_pos_fixed] = 1
        else:
            transitions_map[tuple(positions)][transitions_pos_fixed] += 1
    return transitions, diff_mins, diff_maxs, transitions_map, transitions_min, transitions_max

transitions, _, _, transitions_map, _, _ = get_transitions_map(dfc, [5, 10, 20], 1, [10, 10, 10], 10)
transitions

In [ ]:
transitions_map